# Assignment

Assigment is in the intersection of Named Entity Recognition and Dependency Parsing.

0. Evaluate spaCy NER on CoNLL 2003 data (provided)
    - report token-level performance (per class and total)
        - accuracy of correctly recognizing all tokens that belong to named entities (i.e. tag-level accuracy) 
    - report CoNLL chunk-level performance (per class and total);
        - precision, recall, f-measure of correctly recognizing all the named entities in a chunk per class and total  

1. Grouping of Entities.
Write a function to group recognized named entities using `noun_chunks` method of [spaCy](https://spacy.io/usage/linguistic-features#noun-chunks). Analyze the groups in terms of most frequent combinations (i.e. NER types that go together).

2. One of the possible post-processing steps is to fix segmentation errors.
Write a function that extends the entity span to cover the full noun-compounds. Make use of `compound` dependency relation.

# CoNLL Data
From https://www.clips.uantwerpen.be/conll2003/ner/

The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on four types of named entities: persons, locations, organizations and names of miscellaneous entities that do not belong to the previous three groups. 

The CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on a separate line and there is an empty line after each sentence. The first item on each line is a word, the second a part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags and the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only if two phrases of the same type immediately follow each other, the first word of the second phrase will have tag B-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase.


In [1]:
import conll
import my_ents
from sklearn.metrics import classification_report
import pandas as pd
from collections import defaultdict

# token, POS tag, syntactic chunk tag (IOB), entity tag (IOB)
test = conll.read_corpus_conll('dataset/test.txt')
test = [my_ents.Sentence(sent) for sent in test
        if '-DOCSTART-' not in sent[0][0]]
test = test[:100]

# 0. Evaluate spaCy NER on CoNLL 2003 data (provided)
- report token-level performance (per class and total)
    - accuracy of correctly recognizing all tokens that belong to named entities (i.e. tag-level accuracy) 
- report CoNLL chunk-level performance (per class and total);
    - precision, recall, f-measure of correctly recognizing all the named entities in a chunk per class and total

Conversion from Ontonotes tags to CoNLL format

In [2]:
# conversion of tags from Ontonotes (spacy) to CoNLL format
def from_spacy_to_conll(predictions_spacy):
    switcher = {
                ' ': '',
                '': '',
                'ORG': '-ORG',
                'PER': '-PER',
                'LOC': '-LOC',
                'PERSON': '-PER',
                'GPE': '-LOC'
            }

    # LOC, PER, ORG, MISC
    predictions = []

    for sent in predictions_spacy:
        new = []

        for ent in sent:
            # merge iob and entity type
            new.append((ent.text, ent.ent_iob_ + switcher.get(ent.ent_type_,
                                                              '-MISC')))

        predictions.append(new)

    return predictions

## SpaCy predictions with custom tokenizer
Define custom tokenizer for spacy, otherwise spaCy will tokenize differently from how the CoNLL dataset has been tokenized. This would produce different tokens in output, rendering impossible to compute the accuracy.
Because the prediction of each sentence is indipendent of the other sentences, so I used the Pool function to parallelize the task across multiple processes to performe the computation faster. However, for convinience I used a test set of only 100 samples during development, and that case is faster to not use multi processing.

In [3]:
%%time

import spacy
from multiprocessing import Pool

nlp = spacy.load('en_core_web_sm')

def tokenizer_(sent):
    return spacy.tokens.Doc(nlp.vocab, sent.split())

nlp.tokenizer = tokenizer_

if len(test) > 100:
    with Pool(4) as p:
        # spacy predictions, multi-process
        predictions_spacy = p.map(nlp, [str(sent) for sent in test])
else:
    # spacy predictions
    predictions_spacy = [nlp(str(sent)) for sent in test]

[(ent.text, ent.ent_iob_, ent.ent_type_) for ent in predictions_spacy[5][0:3]]

CPU times: user 1.54 s, sys: 38.7 ms, total: 1.58 s
Wall time: 1.58 s


[('China', 'B', 'GPE'), ('controlled', 'O', ''), ('most', 'O', '')]

In [4]:
# convert to NLTK format so that conll.evaluate can be used
predictions = from_spacy_to_conll(predictions_spacy)

predictions[5][0:3]

[('China', 'B-LOC'), ('controlled', 'O'), ('most', 'O')]

## Token-level accuracy, total and per-class

In [5]:
# organize test data in tuples (entity, tag)
test_set = [[(ent.text, ent.ent_tag) for ent in sent.ents] for sent in test]

test_set[5][0:3]

[('China', 'B-LOC'), ('controlled', 'O'), ('most', 'O')]

### Total accuracy

In [6]:
def total_accuracy(predictions_labels, labels):
    if len(predictions_labels) != len(labels):
        raise Exception('Prediction labels and test labels have different lenght')

    correct = 0
    for i in range(len(predictions_labels)):
        if predictions_labels[i] == labels[i]:
            correct += 1

    return correct/len(labels)


pred_labels = [ent[1] for sent in predictions for ent in sent]
test_labels = [ent[1] for sent in test_set for ent in sent]

round(total_accuracy(pred_labels, test_labels), 3)

0.797

### Per-class accuracy

In [7]:
per_class_metrics = classification_report(test_labels, pred_labels,
                                          output_dict=True)
pd_tbl_class = pd.DataFrame().from_dict(per_class_metrics).transpose()
pd_tbl_class.round(decimals=3)

,precision,recall,f1-score,support
B-LOC,0.840,0.768,0.803,82.000
B-MISC,0.086,0.545,0.148,22.000
B-ORG,0.133,1.000,0.235,2.000
B-PER,0.958,0.622,0.754,111.000
I-LOC,1.000,0.889,0.941,9.000
I-MISC,0.098,0.500,0.164,12.000
I-ORG,0.059,1.000,0.111,1.000
I-PER,0.873,0.711,0.784,97.000
O,0.946,0.831,0.885,1086.000
accuracy,0.797,0.797,0.797,0.797


### Chunk-level accuracy

In [8]:
results = conll.evaluate(test_set, predictions)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
LOC,0.840,0.768,0.803,82
ORG,0.067,0.500,0.118,2
PER,0.903,0.586,0.710,111
MISC,0.079,0.500,0.136,22
total,0.464,0.645,0.539,217


# 1. Grouping of Entities.
Write a function to group recognized named entities using `noun_chunks` method of [spaCy](https://spacy.io/usage/linguistic-features#noun-chunks). Analyze the groups in terms of most frequent combinations (i.e. NER types that go together).

In [9]:
def is_in_a_group(ent, groups):
    for group in groups:
        for elem in group:
            if elem.i == ent.i:
                return True
    return False


def insert_in_group(ent, groups):
    # search the group containing the previous token in sentence order
    for i, group in enumerate(groups):
        for elem in group:
            if elem.i == ent.i - 1:
                # add the ent in next position
                return groups.insert(i+1, [ent])


def group_by_noun_chunk(sent_doc):
    groups = [[ent for ent in chunk if ent.ent_type_ != '']
              for chunk in sent_doc.noun_chunks]
    groups = [group for group in groups if len(group) > 0]

    for ent in sent_doc:
        # check if ent already in a group
        if ent.ent_type_ != '' and not is_in_a_group(ent, groups):
            # insert the missing entity in sentence order
            insert_in_group(ent, groups)

    return groups


predictions_grouped = [group_by_noun_chunk(sent) for sent in predictions_spacy]
print(predictions_spacy[5])
predictions_grouped[5]

China controlled most of the match and saw several chances missed until the 78th minute when Uzbek striker Igor Shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing Chinese keeper and into an empty net . 


[[China], [the, 78th, minute], [Uzbek, Igor, Shkvyrin], [Chinese]]

Swap each token with its tag

In [10]:
def to_tag_groups(grouped_sents):
    for i, sent in enumerate(grouped_sents):
        for j, group in enumerate(sent):
            for k, ent in enumerate(group):
                grouped_sents[i][j][k] = ent.ent_type_


to_tag_groups(predictions_grouped)
predictions_grouped[5]

[['GPE'], ['TIME', 'TIME', 'TIME'], ['NORP', 'PERSON', 'PERSON'], ['NORP']]

## Count frequencies of named entities combinations

In [11]:
def get_group_freqs(grouped_sents):
    freqs = defaultdict(lambda: 0, {})

    for i, sent in enumerate(grouped_sents):
        for j, group in enumerate(sent):
            freqs['-'.join([str(ent) for ent in group])] += 1

    return freqs


pd_freq_tbl = pd.DataFrame().from_dict(get_group_freqs(predictions_grouped),
                                       orient='index', columns=['Count'])
pd_freq_tbl.sort_values(by='Count', ascending=False)

,Count
GPE,63
PERSON-PERSON,38
CARDINAL,24
CARDINAL-PERSON-PERSON,17
DATE,13
ORG,9
NORP,8
TIME-TIME-TIME,7
ORG-ORG,7
TIME-TIME-TIME-TIME,6


# 2. One of the possible post-processing steps is to fix segmentation errors.
Write a function that extends the entity span to cover the full noun-compounds. Make use of `compound` dependency relation.

In [12]:
# for i,sent in enumerate(predictions_spacy):
#     for ent in sent:
#         if ent.dep_ == 'compound' and (ent.ent_type_ != '' or ent.head.ent_type_ != ''):
#             print(i, ent.i)
ts = predictions_spacy[5].copy()
spacy.displacy.render(ts, style='dep')
spacy.displacy.render(ts, style='ent')
for ent in ts:
    if ent.dep_ == 'compound':
        print(f'{ent.text}, {ent.dep_}, {ent.ent_iob_}, {ent.ent_type_}')

Uzbek, compound, B, NORP
striker, compound, O, 
Igor, compound, B, PERSON


In [14]:
def fix_segmentation(spacy_sents):
    # if a token in a noun chunk has compound child that is not in another noun
    # chunk, expand the chunk to include it
    for sent in spacy_sents:
        for span in sent.ents:
            for ent in span:
                for child in ent.children:
                    # the child is a compound for the parent
                    # because it has an IOB tag of O, it is not part of any
                    # other span. So, its parent's span has to be
                    # expandend to include the child
                    if child.dep_ == 'compound' and child.ent_iob_ == 'O':
                        # child on the left of the parent
                        if child.i < ent.i:
                            # find the first token of the chunk and move its B
                            # to the left
                            print(child)


fix_segmentation([ts])
new_span = spacy.tokens.Span(ts, 17, 20, label='PERSON')
print(new_span)
ts.set_ents([new_span], default='unmodified')
spacy.displacy.render(ts, style='ent')
ts.ents

striker
striker Igor Shkvyrin


(China, the 78th minute, Uzbek, striker Igor Shkvyrin, Chinese)